<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/RandomPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title Upgrade to TensorFlow nightly
!pip install --upgrade tf-nightly

     |████████████████████████████████| 397.5MB 42kB/s 
     |████████████████████████████████| 3.8MB 52.4MB/s 
     |████████████████████████████████| 14.5MB 194kB/s 
     |████████████████████████████████| 1.3MB 46.9MB/s 
     |████████████████████████████████| 11.9MB 44.9MB/s 
     |████████████████████████████████| 471kB 51.1MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: grpcio 1.33.2
    Uninstalling grpcio-1.33.2:
      Successfully uninstalled grpcio-1.33.2
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4

In [1]:
import tensorflow as tf

In [2]:
from scipy.special import erf
import numpy as np
import time

In [3]:
x = tf.math.sobol_sample(10,10)

In [20]:
tf.xla.experimental.compile

<function tensorflow.python.compiler.xla.xla.compile>

In [5]:
!pip install numba

In [12]:
from numba import jit
import numpy as np

In [24]:
class BrownianBridge():
    def __init__(self, number_time_steps):
        super(BrownianBridge, self).__init__()

        self.number_time_steps = number_time_steps

        self.left_index = np.zeros(number_time_steps, dtype=int)
        self.right_index = np.zeros(number_time_steps, dtype=int)
        self.bridge_index = np.zeros(number_time_steps, dtype=int)
        self.left_weight = np.zeros(number_time_steps)
        self.right_weight = np.zeros(number_time_steps)
        self.std_dev = np.zeros(number_time_steps)

        self._map = np.zeros(number_time_steps, dtype=int)

        self._map[-1] = 1
        self.bridge_index[0] = number_time_steps - 1
        self.std_dev[0] = tf.math.sqrt(1.0 * number_time_steps)
        self.left_weight[0] = 0
        self.right_weight[0] = 0

        j=0
        for i in range(1,number_time_steps):
            while self._map[j] == True:
                j = j + 1
            k = j
            while self._map[k] == False:
                k = k + 1
            l = j+((k-1-j)>>1)
            self._map[l]=i
            self.bridge_index[i]=l
            self.left_index[i]=j
            self.right_index[i]=k
            self.left_weight[i]=(k-l)/(k+1-j)
            self.right_weight[i]=(1+l-j)/(k+1-j)
            self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
            j=k+1
            if j>=number_time_steps:
                j=0

    @tf.function(jit_compile=True)
    def buildPath(self, z, number):
        path = tf.unstack(tf.zeros(shape=(self.number_time_steps,number)))
        path[-1] = self.std_dev[0]*z[0];
        for i in np.arange(1,self.number_time_steps):
            j = self.left_index[i]
            k = self.right_index[i]
            l = self.bridge_index[i]
            #if j > 0:
            #    path[l] = self.left_weight[i] * path[j-1] + self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
            #else:
            path[l] = self.right_weight[i] * path[k] + self.std_dev[i] * z[i]

        return path

In [208]:
number_time_steps = 128
number = 3

left_index = np.zeros(number_time_steps, dtype=int)
right_index = np.zeros(number_time_steps, dtype=int)
bridge_index = np.zeros(number_time_steps, dtype=int)
left_weight = np.zeros(number_time_steps)
right_weight = np.zeros(number_time_steps)
std_dev = np.zeros(number_time_steps)

_map = np.zeros(number_time_steps, dtype=int)

_map[-1] = 1
bridge_index[0] = number_time_steps - 1
std_dev[0] = tf.math.sqrt(1.0 * number_time_steps)
left_weight[0] = 0
right_weight[0] = 0

j=0
for i in range(1,number_time_steps):
  while _map[j] == True:
    j = j + 1
  k = j
  while _map[k] == False:
    k = k + 1
l = j+((k-1-j)>>1)
_map[l]=i
bridge_index[i]=l
left_index[i]=j
right_index[i]=k
left_weight[i]=(k-l)/(k+1-j)
right_weight[i]=(1+l-j)/(k+1-j)
std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
j=k+1
if j>=number_time_steps:
  j=0

bridge_index_t = tf.constant(bridge_index)
right_index_t = tf.constant(right_index)
left_index_t = tf.constant(left_index)

@tf.function(jit_compile=True)
def buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):
  path = tf.unstack(tf.zeros(shape=(number_time_steps,number),dtype=tf.dtypes.float64))
  path[-1] = std_dev[0]*z[0];
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [216]:
x = tf.math.sobol_sample(128,5,dtype=tf.dtypes.float64)
z = tf.transpose(tf.math.erf(x))

In [217]:
path = buildPath(z, 5, 128, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)

In [212]:
print(tf.autograph.to_code(buildPath.python_function))

def tf__buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):
    with ag__.FunctionScope('buildPath', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        path = ag__.converted_call(ag__.ld(tf).unstack, (ag__.converted_call(ag__.ld(tf).zeros, (), dict(shape=(ag__.ld(number_time_steps), ag__.ld(number)), dtype=ag__.ld(tf).dtypes.float64), fscope),), None, fscope)
        ag__.ld(path)[(- 1)] = (ag__.ld(std_dev)[0] * ag__.ld(z)[0])
        j = 0
        k = 0
        l = 0
        i = 0

        def get_state():
            return ()

        def set_state(block_vars):
            pass

        def loop_body(itr):
            i = itr
            j = ag__.ld(left_index)[ag__.ld(i)]
            k = ag__.ld(right_index)[ag__.ld(i)]
            l = ag__.ld(bridge

In [213]:
bridge = BrownianBridge(2)

In [214]:
x = tf.math.sobol_sample(2,3)

z = tf.transpose(tf.math.erf(x))

path = bridge.buildPath(z, 3)

In [258]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  128#@param {type:"integer"}
number =  8#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = buildPath(z, number, 128, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = buildPath(z, number, 128, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
    print(w[0])
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.00442194938659668
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
Second time on a CPU:  0.003204345703125


In [234]:
print(tf.autograph.to_code(bridge.buildPath.python_function))

def tf__bound_method_wrapper(*args, **kwargs):
    'Wraps either a dummy MethodType or a converted AutoGraph function.'
    with ag__.FunctionScope('bound_method_wrapper', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        strong_bound_method_wrapper = ag__.converted_call(ag__.ld(weak_bound_method_wrapper), (), None, fscope)
        wrapped_fn = ag__.ld(strong_bound_method_wrapper).__wrapped__

        def get_state_1():
            return (do_return, retval_, wrapped_fn)

        def set_state_1(vars_):
            nonlocal retval_, do_return, wrapped_fn
            (do_return, retval_, wrapped_fn) = vars_

        def if_body_1():
            nonlocal retval_, do_return, wrapped_fn
            wrapped_fn = ag__.ld(original_function).python_function

            def get_state():
                return (wrapped_fn,)

     

In [21]:
from tensorflow.contrib.compiler import xla

ModuleNotFoundError: ignored

In [37]:
@tf.function(jit_compile=True)
def f(x,y):
  return x+y

In [40]:
z = f(2,3)

In [41]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x, y):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = (ag__.ld(x) + ag__.ld(y))
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [53]:
@tf.function(jit_compile=True)
def buildPath(z, number, steps):
  x = 0
  for i in range(number):
    x = x + z[steps]
#        path = tf.unstack(tf.zeros(shape=(self.number_time_steps,number)))
#        path[-1] = self.std_dev[0]*z[0];
#        for i in np.arange(1,self.number_time_steps):
#            j = self.left_index[i]
#            k = self.right_index[i]
#            l = self.bridge_index[i]
#            #if j > 0:
#            #    path[l] = self.left_weight[i] * path[j-1] + self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
#            #else:
#            path[l] = self.right_weight[i] * path[k] + self.std_dev[i] * z[i]


In [54]:
x = tf.math.sobol_sample(2,3)

In [55]:
buildPath(x, 1, 1)

In [56]:
print(tf.autograph.to_code(buildPath.python_function))

def tf__buildPath(z, number, steps):
    with ag__.FunctionScope('buildPath', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        x = 0

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body(itr):
            nonlocal x
            i = itr
            x = (ag__.ld(x) + ag__.ld(z)[ag__.ld(steps)])
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(number),), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})



In [50]:
import torch

In [60]:
@torch.jit.script
def foo(x, number):
  x = 0
  for i in range(number):
    x = x + z[i]

# See the compiled graph as Python code
print(foo.code)

# Call the function using the TorchScript interpreter
foo(x , 2)

RuntimeError: ignored